<a href="https://colab.research.google.com/github/AnastasiaBlack/w20061036_Anastasiia_Ilina_Landmines_Detection_Research/blob/main/colabFiles/YOLOv8_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# New Section

In [ ]:
#Memory info
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

https://docs.ultralytics.com/modes/predict/#plotting-results

Train the preexisting model (https://docs.ultralytics.com/datasets/detect/#ultralytics-yolo-format)

In [ ]:
!pip install ultralytics
from ultralytics import YOLO
import time

start_time = time.time()  # Start time

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data='/content/drive/MyDrive/University/Research_Module/dataset/Yolov8_v3_dataset/Landmines detection dataset.v3i.yolov8/data.yaml', epochs=200, imgsz=[336,256], rect=True)

end_time = time.time()  # End time

training_time = end_time - start_time
print(f"Training time: {training_time} seconds")


In [ ]:
# Validate

from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load an official model
model = YOLO('/content/runs/detect/train2/weights/best.pt')  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category
print(metrics)

In [ ]:
# !pip install ultralytics
from PIL import Image
# Prediction
from ultralytics import YOLO
import os
labels=["landmines"]

# Load a pretrained YOLOv8n-cls Classify model
model = YOLO('/content/runs/detect/train2/weights/best.pt')

test_imgs_folder="/content/drive/MyDrive/University/Research_Module/dataset/Yolov8_v3_dataset/Landmines detection dataset.v3i.yolov8/test/images"
# test_imgs_folder="/content/try"
result_folder = '/content/detection_results_custom_2/'
os.makedirs(result_folder, exist_ok=True)
os.makedirs(result_folder + 'images/', exist_ok=True)
for image_path in os.listdir(test_imgs_folder):
  # Run inference on an image
  results = model(os.path.join(test_imgs_folder, image_path))

  for r in results:
      if len(r.boxes)==0:
        print("File has no detectioins: ", image_path)
      for i in range(len(r.boxes)):
        class_id = int(r.boxes.cls[i].item())
        confidence = int(100 * float(r.boxes.conf[i].item()))
        left, bottom, right, top = map(int, r.boxes.xyxy[i].cpu().numpy().tolist())  # Remove [0]

        # METRICS FILES PREPARATION "/content/detection_result/": Generate the output file path with the same name as the input XML file but with a ".txt" extension

        output_file_path = os.path.join(result_folder, os.path.splitext(os.path.basename(image_path))[0] + ".txt")
        with open(output_file_path, "w") as f:
          f.write(f"{labels[class_id]} {confidence} {left} {bottom} {right} {top}\n")

          # line = f"{class_id} {confidence:.4f} {left:.4f} {bottom:.4f} {right:.4f} {top:.4f}\n"
          # print(line)
      im_array = r.plot()  # plot a BGR numpy array of predictions
      im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
      image_id = image_path.split('_')[0]
      im.save(result_folder + 'images/' + image_id+'_results'+'.jpg')  # save image

In [ ]:
# To get speed for all images:
import time
start_time = time.time()
results = model(test_imgs_folder)
end_time = time.time()

testing_time = end_time-start_time
print('Testing time: ', testing_time)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

When it comes to evaluating trained YOLOv8 models with test data where ground truth is available, the built-in Val mode in YOLOv8 generally offers the most reliable approach. This mode provides precise metrics—including precision, recall, mAP@.5, mAP@.5:.95—and chart visualization of training versus validation metrics for proper understanding of models' performances.

In [ ]:
# !pip install ultralytics

# from ultralytics import YOLO

# model = YOLO('/content/best.pt')
results = model.val(
    data='/content/drive/MyDrive/University/Research_Module/dataset/Phase2_YOLOv8.v4i.yolov8/data.yaml',
    split='test'
)
results.box.map    # map50-95
results.box.map50  # map50
results.box.map75  # map75
results.box.maps   # a list contains map50-95 of each category


In [ ]:
print(results.confusion_matrix.tp_fp())

Let's count true positives, false positives and false negatives: https://docs.ultralytics.com/reference/utils/metrics/#ultralytics.utils.metrics.ConfusionMatrix.process_cls_preds

In [ ]:
def tp_fp(confusion_matrix):
    """Returns true positives and false positives."""
    tp = confusion_matrix.matrix.diagonal()  # true positives
    fp = confusion_matrix.matrix.sum(1) - tp  # false positives
    fn = confusion_matrix.matrix.sum(0) - tp  # false negatives (missed detections)
    return (tp[:-1], fp[:-1], fn[:-1]) if confusion_matrix.task == 'detect' else (tp, fp, fn)  # remove background class if task=detect

tp, fp, fn = tp_fp(results.confusion_matrix)
print('True positive ', tp[0])
print('False positive ', fp[0])
print('False negative ', fn[0])


In [ ]:
print('Precision: ' , results.box.mp)
print('Recall: ' , results.box.mr)
print('F1 score: ' , results.box.f1)

In [ ]:
# Analyze the results to count false negatives
false_negatives_count = 0
# for result in results:
print(results)
    # for pred in result['pred']:
    #     if not pred:  # If no predictions were made
    #         false_negatives_count += 1


In [ ]:
# ZIP folder with all the content
import shutil

# 1) Create a zip file for default test validation results with curves pngs:
shutil.make_archive('/content/yolov8_detection_runs_val', 'zip', '/content/runs')
from google.colab import files
files.download('/content/yolov8_detection_runs_val.zip')


# 2) Create a zip file for custom detection txt files with bounding boxes:
shutil.make_archive('/content/detection_results_custom_2', 'zip', '/content/detection_results_custom_2')

from google.colab import files
files.download('/content/detection_results_custom_2.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil
from google.colab import files

# 3) Create a zip file for train results:
shutil.make_archive('/content/runs/detect/train', 'zip', '/content/runs/detect/train')

from google.colab import files
files.download('/content/runs/detect/train.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>